In my last [notebook](https://www.kaggle.com/afajohn/h2o-individual-models-vs-positive-counts) I stated labels with higher number of positive labels contributed negatively to the overall log loss when a model is trained for each label. 

In this notebook I selected one the poor performing labels to see whether label based feature selection on gene columns will improve the results. 

Thanks to [@aerdem4](https://www.kaggle.com/aerdem4) for LOFO Importance library. 

Building 206 models is a long way to approach to problem, each model with their own search space with hyperparameters and selected features. However for some of the labels single multilabel model predcitions can be "boosted" by zooming into some of the labels with single models.

In [ ]:
! pip install skorch==0.8.0
! pip install lofo-importance

In [ ]:
import pandas as pd
from sklearn.model_selection import KFold
from lofo import LOFOImportance, Dataset, plot_importance, FLOFOImportance
from sklearn.model_selection import StratifiedKFold, StratifiedShuffleSplit, train_test_split
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMRegressor
from sklearn.metrics import log_loss
import numpy as np
%matplotlib inline

In [ ]:
train = pd.read_csv('../input/lish-moa/test_features.csv')
train = train[train.cp_type!='ctl_vehicle']
train_labels = pd.read_csv('../input/lish-moa/train_targets_scored.csv')

In [ ]:
gene_cols = [col for col in train.columns if 'g-' in col]
labels = train_labels.columns.difference(['sig_id']).tolist()
train_df = train.drop(['sig_id'], axis=1)[gene_cols+['cp_time']]

In [ ]:
label_df = train_labels[labels]

In [ ]:
df = pd.concat([train_df,label_df],axis=1).dropna().reset_index(drop=True)

In [ ]:
l = 79
label_df.iloc[:,l].value_counts()

In [ ]:
sss = StratifiedShuffleSplit(n_splits=2,test_size=0.28, random_state=42)
sp1,sp2 = sss.split(df[[labels[l]]+['cp_time']].values,df[[labels[l]]+['cp_time']].values)

In [ ]:
train_df = df.iloc[list(sp1[0]),:].reset_index(drop=True)
val_df = df.iloc[list(sp1[1]),:].reset_index(drop=True)

In [ ]:
rf = RandomForestClassifier(n_jobs=-1, max_depth=6)
rf.fit(train_df[gene_cols],train_df[labels[l]])

In [ ]:
lofo_imp = FLOFOImportance(trained_model=rf, validation_df=val_df,features=gene_cols,target=labels[l],scoring='neg_log_loss',n_jobs=-1)

In [ ]:
imps = lofo_imp.get_importance()

In [ ]:
plot_importance(imps[:40])

In [ ]:
plot_importance(imps[-40:])

### Experiments with Important Features

#### Full Set

In [ ]:
label = label_df.columns[l]

In [ ]:
X,y = df[gene_cols], df[label]

In [ ]:
skf = StratifiedKFold(n_splits=4, shuffle=True, random_state=1881).split(y,y)
scores = []

for i, (tr_ix, vl_ix) in enumerate(skf):
    print(f'================Fold-{i+1}===================')
    
    train_X, train_y = X.iloc[tr_ix], y.iloc[tr_ix]
    valid_X, valid_y = X.iloc[vl_ix], y.iloc[vl_ix]
    
    model = RandomForestClassifier(max_depth=6, n_jobs=-1, n_estimators=300)
    
    model.fit(train_X, train_y)
    
    preds = model.predict_proba(valid_X)[:,1]
    
    score = log_loss(valid_y, preds)
    
    scores.append(score)
    
print(np.mean(scores))

#### Reduce Features

In [ ]:
elim = 40
reduced_feats = imps.feature.values[:elim].tolist()

In [ ]:
X,y = df[reduced_feats], df[label]

In [ ]:
skf = StratifiedKFold(n_splits=4, shuffle=True, random_state=1881).split(y,y)
scores = []

for i, (tr_ix, vl_ix) in enumerate(skf):
    print(f'================Fold-{i+1}===================')
    
    train_X, train_y = X.iloc[tr_ix], y.iloc[tr_ix]
    valid_X, valid_y = X.iloc[vl_ix], y.iloc[vl_ix]
    
    model = RandomForestClassifier(max_depth=6, n_jobs=-1, n_estimators=300)
    
    model.fit(train_X, train_y)
    
    preds = model.predict_proba(valid_X)[:,1]
    
    score = log_loss(valid_y, preds)
    
    scores.append(score)
    
print(np.mean(scores))

### NN

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset

import skorch
from skorch import NeuralNetBinaryClassifier

In [ ]:
class Net(nn.Module):
    def __init__(self, input_size):
        super(Net, self).__init__()
        self.input_size = input_size
        
        self.input = nn.Linear(self.input_size, 512)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.4)
        self.hidden1 = nn.Linear(512,256)
        self.hidden2 = nn.Linear(256,128)
        self.out = nn.Linear(128,1)
        self.bn1 = nn.BatchNorm1d(512)
        self.bn2 = nn.BatchNorm1d(256)
        self.bn3 = nn.BatchNorm1d(128)
        self.sig = nn.Sigmoid()
        
    def forward(self, X):
        
        x = self.input(X)
        x = self.relu(x)
        x = self.bn1(x)
        x = self.dropout(x)
        
        x = self.hidden1(x)
        x = self.relu(x)
        x = self.bn2(x)
        x = self.dropout(x)
        
        x = self.hidden2(x)
        x = self.relu(x)
        x = self.bn3(x)
        x = self.dropout(x)
        
        x = self.out(x)
        out = self.sig(x)
        
        return out
        
        

In [ ]:
X,y = df[gene_cols], df[label]

In [ ]:
skorch.__version__

In [ ]:
skf = StratifiedKFold(n_splits=4, shuffle=True, random_state=1881)
model = NeuralNetBinaryClassifier(Net(X.shape[1]),
                                      optimizer=torch.optim.Adam,
                                      train_split=skorch.dataset.CVSplit(skf),
                                      max_epochs=10,
                                      criterion=torch.nn.modules.loss.BCELoss)

In [ ]:
model.fit(torch.Tensor(X.values),torch.Tensor(y.values))

In [ ]:
elim = 40
reduced_feats = imps.feature.values[:elim].tolist()

In [ ]:
X,y = df[reduced_feats], df[label]

In [ ]:
skf = StratifiedKFold(n_splits=4, shuffle=True, random_state=1881)
model = NeuralNetBinaryClassifier(Net(X.shape[1]),
                                      optimizer=torch.optim.Adam,
                                      train_split=skorch.dataset.CVSplit(skf),
                                      max_epochs=10,
                                      criterion=torch.nn.modules.loss.BCELoss)

In [ ]:
model.fit(torch.Tensor(X.values),torch.Tensor(y.values))